#### Задание 1.
Построить алгоритм и проргамму для вычисления двукратного интеграла
$$
I = \iint_G f(x, y) dx dy
$$
по области $G$, ограниченной прямыми
$$
x + y = 1, x = 0, y = 0.
$$

Функция $z = f(x, y)$ задана таблично в файле `data.csv`.

Применить метод последовательного интегрирования. По одному направлению использоавть формулу Гаусса, а по другому - формулу Симпсона.

*Указание.* При двумерной интерполяции по таблице функции $z = f(x, y)$ использовать выравнивающие переменные $(\eta, x, y)$, причём $\eta = \ln z$.

**Результаты.**
1. Разработать алгоритм вычисления $n$ корней полинома Лежандра $n$-ой степени $P_n(x)$ при реализации формулы Гаусса.
2. Исследовать влияние количества выбираемых узлов сетки по каждому направлению на результаты расчётов.

In [132]:
import numpy as np
import sympy as sp
import pandas as pd
from typing import Callable as Fn, Dict, Tuple

# Read table
df = pd.read_csv('data.csv')

# x_start, y_start = 0, 0
# x_end, y_end = 1.2, 1.2
# x_points, y_points = 13, 13
n_points = 13

# x = np.linspace(x_start, x_end, x_points, endpoint=True)
# y = np.linspace(y_start, y_end, y_points, endpoint=True)
x = np.array(list(map(lambda x: x / 10, range(n_points))))
y = np.array(list(map(lambda x: x / 10, range(n_points))))
z = df.values

# Alignment variable
eta = np.array([[np.log(i) for i in zi] for zi in z])

def table_fn(x_: float, y_: float, use_eta=True):
    x_ind = list(x).index(x_)
    y_ind = list(y).index(y_)
    
#     print(x_ind, y_ind)
    
    result = eta[y_ind][x_ind] if use_eta else z[y_ind][x_ind]

    return result

def f(x: float):
    return x ** 2

def get_gauss_rhs_coefs_for_system_of_equations(k):
    return 2 / (k + 1) if k % 2 == 0 else 0

# def get_legendre_polynome(n):
#     x_symb = sp.Symbol('x')
#     n_symb = sp.Symbol('n')
#     lhs = 1 / (2**n_symb * sp.factorial(n_symb))
#     rhs = (x_symb**2 - 1)**n_symb
#     lhs = lhs.subs({n_symb: n})
#     rhs = rhs.subs({n_symb: n})
#     lp = lhs * sp.diff(rhs, x_symb, n)
#     legendre_polynome_fn = sp.lambdify(x_symb, lp)
#     return legendre_polynome_fn

from numpy.polynomial.legendre import Legendre
def get_legendre_polynomial_roots(n):
    return Legendre(n * [0] + [1]).roots()

def get_coefficients(legendre_polynomial_roots):
    roots = legendre_polynomial_roots
    n = len(roots)
    
    matrix = np.array([[t ** t_power for t in roots] for t_power in range(n)])

    values = np.array([get_gauss_rhs_coefs_for_system_of_equations(k) for k in range(n)])
    
    coefs = np.linalg.solve(matrix, values)
    
    return coefs
    
def integrate_gauss(func: Fn[[float], float], a: float, b: float, n=3):
    roots = get_legendre_polynomial_roots(n)
    coefs = get_coefficients(roots)
    
    xs = np.array(list(map(lambda t: (b - a) / 2 * t + (a + b) / 2, roots)))
    
    result = (b - a) / 2 * sum([coefs[i] * func(xs[i]) for i in range(len(roots))])

    return result

def integrate_simpson(func: Fn[[float], float], a: float, b: float, n=30):
    h = (b - a) / (n*2)
    
    xs = np.array([a + i * h for i in range(n*2 + 1)])
    ys = np.array([func(x) for x in xs])
    
    s1 = 4 * np.sum([ys[2*i - 1] for i in range(1, n)])
    s2 = 2 * np.sum([ys[2*i] for i in range(1, n - 1)])
    s = (h / 3) * (ys[0] + s1 + s2 + ys[-1])
    
    return s

# Integration region
def G(x: float, y: float):
    return x >= 0 and y >= 0 and (x + y) <= 1

IntegrationFn = Fn[[Fn[[float], float], float, float], float]
Point = Tuple[float, float]
RegionDict = Dict[Point, float]
from scipy.interpolate import CubicSpline as Spline
def integrate_region(integrate_main: IntegrationFn,
                     integrate_Fs: IntegrationFn,
                     region_dict: RegionDict,
                     use_eta = False
                     ):
    xs = np.unique(list(map(lambda p: p[0], region_dict.keys())))
    ys = np.unique(list(map(lambda p: p[1], region_dict.keys())))
    
    points_by_y = lambda y: [p for p in region_dict.keys() if p[1] == y]
    
    Fs = []
    Ys = []
    
    for k in ys:
        points = points_by_y(k)
        xs = [p[0] for p in points]
        fn = [region_dict[p] for p in points]
        
        if len(xs) > 1:
            spline = Spline(xs, fn)
            
            if use_eta:
                func = lambda x: np.e ** spline(x)
            else:
                func = lambda x: spline(x)
            
            x_min = min(xs)
            x_max = max(xs)
        
            Fk = integrate_Fs(func, x_min, x_max)
            
            Fs.append(Fk)
            Ys.append(k)
    
    func = Spline(Ys, Fs)
    
    y_min = min(Ys)
    y_max = max(Ys)
    
    return integrate_main(func, y_min, y_max)

# print(integrate_gauss(f, 2, 5))

# G = lambda x, y: True
# x = np.arange(0, 1.1, 0.1)
# y = np.arange(0, 1.1, 0.1)
# table_fn = lambda x, y: 1
g_points = [(x_, y_) for y_ in y for x_ in x if G(x_, y_)]

g_dict = dict(zip(g_points, [table_fn(*p, use_eta=False) for p in g_points]))
# g_dict = dict(zip(g_points, [table_fn(*p) for p in g_points]))

# i0 = integrate_region(integrate_gauss, integrate_gauss, g_dict)
# i1 = integrate_region(integrate_gauss, integrate_simpson, g_dict)
# i2 = integrate_region(integrate_simpson, integrate_simpson, g_dict)
i3 = integrate_region(integrate_simpson, integrate_gauss, g_dict)

# integrals_z = [i0, i1, i2, i3]

print(i3)

g_dict = dict(zip(g_points, [table_fn(*p, use_eta=True) for p in g_points]))
# g_dict = dict(zip(g_points, [table_fn(*p) for p in g_points]))

# i0 = integrate_region(integrate_gauss, integrate_gauss, g_dict)
# i1 = integrate_region(integrate_gauss, integrate_simpson, g_dict)
# i2 = integrate_region(integrate_simpson, integrate_simpson, g_dict)
i3 = integrate_region(integrate_simpson, integrate_gauss, g_dict, use_eta = True)

# integrals_eta = [i0, i1, i2, i3]

print(i3)

0.5754248749988361
0.575420655477609


#### Задание 2.

Задана табличная (сеточная) функция. Имеется информация, что закономерность, представленная этой таблицей, может быть описана формулой
$$
y = \frac{a_0 x}{a_1 + a_2 x},
$$
параметры функции неизвестны **и определять их не нужно**.

| x | y | 1 | 2 | 3 | 4 | 5 |
| - | - | - | - | - | - | - |
| 1 | 0.571 |   |   |   |   |   |
| 2 | 0.889 |   |   |   |   |   |
| 3 | 1.091 |   |   |   |   |   |
| 4 | 1.231 |   |   |   |   |   |
| 5 | 1.333 |   |   |   |   |   |
| 6 | 1.412 |   |   |   |   |   |

Вычислить первые разностные производные от функции и занести их в стоблцы (1)-(4) таблицы:
* 1 - односторонняя разностная производная,
* 2 - центральная разностная производная,
* 3 - 2-я формула Рунге с использованием односторонней производной,
* 4 - введены выравнивающие переменные
* 5 - вторая разностная производная

**Результаты.**
Заполненная таблица с краткими комментариями по поводу используемых формул и их точности.